In [ ]:
%load_ext autoreload
%autoreload 2
%pip install colour ruptures
import pandas as pd
import numpy as np
import bz2
import csv
import io
import json
import re
import time
import random
import requests
import datetime
from tqdm import tqdm
import json
from matplotlib import ticker
from colour import Color
import ruptures as rpt
from pathlib import Path
from pprint import pprint
from typing import List, Dict
from dateutil.relativedelta import relativedelta
import lsde2021.csv as csvutils
import lsde2021.utils as utils
import lsde2021.download as dl
import lsde2021.changepoints as cp
from pyspark.sql import SparkSession
import pyspark.sql.types as T
import pyspark.sql.functions as F

import matplotlib.pyplot as plt

plt.rcParams["figure.figsize"] = (20,3)

In [ ]:
MAX_MEMORY = "30G"

spark = SparkSession \
    .builder \
    .appName("parse-wikipedia-sql-dumps") \
    .config("spark.executor.memory", MAX_MEMORY) \
    .config("spark.driver.memory", MAX_MEMORY) \
    .config('spark.driver.maxResultSize', MAX_MEMORY) \
    .config('spark.ui.showConsoleProgress', 'false') \
    .getOrCreate()
sc = spark.sparkContext

In [ ]:
languages = pd.read_csv("./data/languages.csv", index_col="code")
print(languages["group"].unique().tolist())
languages.head()

In [ ]:
stringency = spark.read.format("parquet").load(f"../nvme/oxcgrt-covid-policy-tracker/OxCGRT_withnotes.parquet")
all_days = list(dl.date_range( datetime.date(2019, 1, 1), datetime.date(2019, 12, 31) ))
all_days += list(dl.date_range( datetime.date(2020, 1, 1), datetime.date(2020, 12, 31) ))
all_days += list(dl.date_range( datetime.date(2021, 1, 1), datetime.date(2021, 12, 31) ))
# print(stringency.select("CountryName").distinct().rdd.flatMap(lambda x: x).collect())

In [ ]:
for country_group in languages["group"].unique().tolist():
    print(country_group)
    year_df = None
    for day in tqdm(all_days):
        data = Path("../nvme/pageview_complete_per_topic") / Path("/".join(dl.wikimedia_pageview_complete_local_file(day, monthly=False))).with_suffix(".parquet")
        df = spark.read.format("parquet").load(str(data / f"group={country_group}"))
        # df = df.filter(F.col(f"level{level}_daily_total").isNotNull())
        df = df.withColumn("date", F.lit(day))

        if year_df is None:
            year_df = df
        else:
            year_df = year_df.union(df)
    year_df.write.format("parquet").mode("overwrite") \
        .save(f"../nvme/country_pageviews/pageviews_{country_group}.parquet")

In [ ]:
test = "Germany"
# test = "China"
test = "Italy"
country_stringency, changepoints = cp.get_change_points(stringency, test)
cp.plot_changepoints(
    country_stringency.toPandas().set_index("Date"),
    changepoints,
    title=f"Changepoints based on stringency index for {test}"
)

In [ ]:
china = stringency.filter(
    (F.lower(F.col("CountryName")) == country.lower())
)
# & (F.col("Date") == F.lit(datetime.date(2020,1,30)))
china = china.groupBy("Date", "CountryName").agg(F.mean("StringencyIndex").alias("StringencyIndex"))
china = china.select("Date", "StringencyIndex")
china = china.sort(F.col("Date").asc())
# , "CountryCode", "RegionName", "RegionCode", "Jurisdiction")
china.show()
china = china.toPandas().set_index("Date")
print(china.head())
plt.plot(china, color="blue", linestyle="-")
plt.show()